In [1]:
import pandas as pd
import numpy as np
import ordpy
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv("cic.csv", sep=";")
df

,Unnamed: 0,total_pacotes,total_pacotes_icmp,total_pacotes_udp,total_pacotes_tcp,maior_pacote,menor_pacote,soma_pacotes,total_ips_origem,total_ips_destino,porta_origem_mais_frequente,porta_destino_mais_frequente,has_bot
0,0,4,0,0,0,94,64,346,2,2,0.0,0.0,0
1,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2,2,31,0,0,18,342,60,2832,8,8,0.0,0.0,0
3,3,6,4,0,0,130,60,456,3,4,0.0,0.0,1
4,4,28,0,0,12,223,60,2938,6,8,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,2019,46,0,0,36,150,60,3732,11,12,0.0,0.0,1
2020,2020,69,4,0,54,100,60,5068,15,14,0.0,0.0,1
2021,2021,54,0,0,48,150,60,4032,13,13,0.0,0.0,1
2022,2022,56,0,0,50,112,60,3960,11,11,0.0,0.0,0


# Loop para extrair as métricas sem variar o taux

In [3]:
matrix = []
label = []
cols = ['total_ips_origem','total_ips_destino', 'total_pacotes', 
        'total_pacotes_icmp','total_pacotes_tcp'
       ]
print(cols)
taux = 1
dx = 3

init = 0
end = 101 
for i in range (1, end):
    matrix.append([[init, i]])

for i in range(0,1924):
    vec = []
    vec.append([init, end])
    
    for col in cols:
        tmp = ordpy.complexity_entropy(df[col][init:end], dx=dx, taux=taux)
        vec.append(tmp[0])
        vec.append(tmp[1])
    
        tmp = ordpy.fisher_shannon(df[col][init:end], dx=dx, taux=taux)
        vec.append(tmp[1])
    
        edges = ordpy.ordinal_network(df[col][init:end], dx=dx, taux=taux)
    
        tmp = ordpy.complexity_entropy(edges[2], dx=dx, taux=taux)
        vec.append(tmp[0])
        vec.append(tmp[1])
    
        tmp = ordpy.fisher_shannon(edges[2], dx=dx, taux=taux)
        vec.append(tmp[1])
    
        tmp = len(edges[1])
        vec.append(tmp)
    
        j=0
        indices = []
        for array in edges[1]:
            if(array[0] == array[1]):
                indices.append(j)
            j += 1
    
        prob = 0
        for index in indices:
            tmp = edges[2][index]
            prob += tmp
        vec.append(prob)

    matrix.append(vec)

    bot = 0
    for line in df.iloc[init:end].iterrows():
        if(line[1][4] == 1):
            bot = 1
    label.append(bot)
        
    init = init + 1
    end = end + 1

['total_ips_origem', 'total_ips_destino', 'total_pacotes', 'total_pacotes_icmp', 'total_pacotes_tcp']


In [4]:
ft = pd.DataFrame(matrix, columns = ['time',
                                     'IPO_entropy', 'IPO_complexity' , 'IPO_fisher_shanon', 'IPO_edge_entropy', 'IPO_edge_complexity', 'IPO_edge_fisher_shannon', 'IPO_number_nodes', 'IPO_self_transition',
                                     'IPD_entropy', 'IPD_complexity' , 'IPD_fisher_shanon', 'IPD_edge_entropy', 'IPD_edge_complexity', 'IPD_edge_fisher_shannon', 'IPD_number_nodes', 'IPD_self_transition',
                                     'TP_entropy', 'TP_complexity' , 'TP_fisher_shanon', 'TP_edge_entropy', 'TP_edge_complexity', 'TP_edge_fisher_shannon', 'TP_number_nodes', 'TP_self_transition',
                                     'TP_ICMP_entropy', 'TP_ICMP_complexity' , 'TP_ICMP_fisher_shanon', 'TP_ICMP_edge_entropy', 'TP_ICMP_edge_complexity', 'TP_ICMP_edge_fisher_shannon', 'TP_ICMP_number_nodes', 'TP_ICMP_self_transition',
                                     'TP_TCP_entropy', 'TP_TCP_complexity' , 'TP_TCP_fisher_shanon', 'TP_TCP_edge_entropy', 'TP_TCP_edge_complexity', 'TP_TCP_edge_fisher_shannon', 'TP_TCP_number_nodes', 'TP_TCP_self_transition'])
ft
ft.to_csv('po_cic.csv',sep=';')
ft[0:50]

,time,IPO_entropy,IPO_complexity,IPO_fisher_shanon,IPO_edge_entropy,IPO_edge_complexity,IPO_edge_fisher_shannon,IPO_number_nodes,IPO_self_transition,IPD_entropy,...,TP_ICMP_number_nodes,TP_ICMP_self_transition,TP_TCP_entropy,TP_TCP_complexity,TP_TCP_fisher_shanon,TP_TCP_edge_entropy,TP_TCP_edge_complexity,TP_TCP_edge_fisher_shannon,TP_TCP_number_nodes,TP_TCP_self_transition
0,"[0, 1]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[0, 2]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[0, 3]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[0, 4]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[0, 5]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"[0, 6]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"[0, 7]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"[0, 8]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"[0, 9]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"[0, 10]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import confusion_matrix
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
import numpy as np

po = pd.read_csv("po_cic.csv", sep=";")
early_warning = pd.read_csv("cic.csv", sep=";")

colunas = ['total_ips_origem', 'total_ips_destino', 'total_pacotes']

df = po[['IPO_entropy', 'IPO_complexity',
       'IPO_fisher_shanon', 'IPO_edge_entropy', 'IPO_edge_complexity',
       'IPO_edge_fisher_shannon', 'IPO_number_nodes', 'IPO_self_transition',
       'IPD_entropy', 'IPD_complexity', 'IPD_fisher_shanon',
       'IPD_edge_entropy', 'IPD_edge_complexity', 'IPD_edge_fisher_shannon',
       'IPD_number_nodes', 'IPD_self_transition', 'TP_entropy',
       'TP_complexity', 'TP_fisher_shanon', 'TP_edge_entropy',
       'TP_edge_complexity', 'TP_edge_fisher_shannon', 'TP_number_nodes',
       'TP_self_transition',
       'TP_ICMP_entropy', 'TP_ICMP_complexity' , 'TP_ICMP_fisher_shanon', 'TP_ICMP_edge_entropy', 'TP_ICMP_edge_complexity', 'TP_ICMP_edge_fisher_shannon', 'TP_ICMP_number_nodes', 'TP_ICMP_self_transition',
       'TP_TCP_entropy', 'TP_TCP_complexity' , 'TP_TCP_fisher_shanon', 'TP_TCP_edge_entropy', 'TP_TCP_edge_complexity', 'TP_TCP_edge_fisher_shannon', 'TP_TCP_number_nodes', 'TP_TCP_self_transition']]
label = early_warning['has_bot']

init = 101
train = 674
end = 1484

x_train = df[init:train]
x_test = df[train:end]
y_real = label[train:end]

clf = OneClassSVM(gamma='auto', kernel='poly', nu=0.1).fit(x_train)
y_test = clf.predict(x_test)
y_test_final = [0 if i == 1 else 1 for i in y_test]
acc = accuracy_score(y_real, y_test_final)
print(acc)
print(confusion_matrix(y_real, y_test_final))
print(classification_report(y_real, y_test_final, digits = 4))

0.6925925925925925
[[560   0]
 [249   1]]
              precision    recall  f1-score   support

           0     0.6922    1.0000    0.8181       560
           1     1.0000    0.0040    0.0080       250

    accuracy                         0.6926       810
   macro avg     0.8461    0.5020    0.4130       810
weighted avg     0.7872    0.6926    0.5681       810

